In [ ]:
# TODO: add back in Silvana

In [45]:
import os
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn import metrics
from tqdm import tqdm
import re

os.chdir(r"C:\Users\payto\Google Drive\01 - Research\McNally Lab Qualitative Coding Projects\Payton's Project\Qualitative Coding\Data")


In [18]:
# Load and clean ratings

claire = pd.read_csv(r"Codesheets - Events - Claire.csv")
margot = pd.read_csv("Codesheets - Events - Margot.csv").iloc[:,3:]
melissa = pd.read_csv("Codesheets - Events - Melissa.csv").iloc[:,3:]
nicole = pd.read_csv("Codesheets - Events - Nicole.csv").iloc[:,3:]
noah = pd.read_csv("Codesheets - Events - Noah.csv").iloc[:,3:]
payton = pd.read_csv("Codesheets - Events - Payton.csv").iloc[:,3:]
#silvana = pd.read_csv("Codesheets - Events - Silvana.csv").iloc[:,3:]

main = claire.iloc[:,:3]
claire = claire.iloc[:,4:]
claire.rename(columns = lambda x: x + "_claire", inplace=True)
margot.rename(columns = lambda x: x + "_margot", inplace=True)
melissa.rename(columns = lambda x: x + "_melissa", inplace=True)
nicole.rename(columns = lambda x: x + "_nicole", inplace=True)
noah.rename(columns = lambda x: x + "_noah", inplace=True)
payton.rename(columns = lambda x: x + "_payton", inplace=True)
#silvana.rename(columns = lambda x: x + "_silvana", inplace=True)

# ADD SILVANA IN LINE BELOW
ratings = pd.concat([main, claire, margot, melissa, nicole, noah, payton], join="inner", axis=1)
ratings.dropna(subset = ['Actual physical injury_noah'], inplace=True)
ratings.rename(columns = lambda x: x.lower(), inplace=True)
ratings.rename(columns = lambda x: re.sub(" ","_",x), inplace=True)

# CHANGE TO 7 for silvana
nRaters = 6

events = ratings.loc[:,'events']
events

0             witnessed a leaf falling from a tree
1                     lost a foot due to frostbite
2                 witnessed a stranger being raped
3                was tackled by a security officer
4        visited family members in another country
                          ...                     
595                  was disowned from a community
596                        was struck by lightning
597    moved to a different house in the same city
598                             smoked a cigarette
599                     visited a holocaust museum
Name: events, Length: 600, dtype: object

# Create features

In [48]:
# Create consensus ratings for each category

categories = ['actual_physical_injury', 'threat_of_death', 'actual_death', 'sexual_content',\
             'physical_pain','human_perpetrator','close_int._perpetrator',\
             'close_int._victim','loss_of_possessions','threat_to_moral_identity',
             'threat_to_social_status_or_other_identity','threat_to_moral_worldview',
             'threat_to_trust_in_interpersonal_relationships']

features = pd.DataFrame(index=range(600), columns=categories)
for cat in categories:
    allRatings = list()
    for i in ratings.columns.values: 
        if bool(re.search(cat, i)):
            allRatings.append(ratings.loc[:,i])
            
    # get averages using pandas
    # columns = 1 for each rater
    allRatingsDF = pd.DataFrame(index=range(600), columns=range(nRaters))
    for i in range(nRaters):
        allRatingsDF.iloc[:,i] = allRatings[i]
    features.loc[:,cat] = round(allRatingsDF.mean(1),0)

# Use regex to add features for learned about, witnessed, happened, child

features['learned_about'] = None
features['witnessed'] = None
features['child'] = None

for i in range(features.shape[0]):
    event = events[i]
    if bool(re.search('learn', event[:10])) or bool(re.search('read', event[:10])):
        features.loc[(features.index == i),'learned_about'] = 1
    else:
        features.loc[(features.index == i),'learned_about'] = 0
    
    if bool(re.search('witness', event[:10])) or bool(re.search('watch', event[:10])):
        features.loc[(features.index == i),'witnessed'] = 1
    else:
        features.loc[(features.index == i),'witnessed'] = 0

    if bool(re.search('child', event)):
        features.loc[(features.index == i),'child'] = 1
    else:
        features.loc[(features.index == i),'child'] = 0
        
# to check regex
#pd.concat([events, features], axis=1).to_csv('check.csv')

# # Add interaction terms
# intr = PolynomialFeatures(interaction_only=True,include_bias = False)
# features = intr.fit_transform(features)

# Create responses

In [20]:
stimuli_means = pd.read_csv(r"Stimuli_Means.csv")
stimuli_means.columns = ["events","means"]

responses = pd.DataFrame(index=range(ratings.shape[0]), columns=["seriousness_mean"])
for i in range(ratings.shape[0]):
    stimIndex = stimuli_means.index[stimuli_means['events'] == ratings.iloc[i, ratings.columns.get_indexer(['events'])][0]].tolist()
    responses.iloc[i,stimuli_means.columns.get_indexer(['seriousness_mean'])[0]] = stimuli_means.iloc[stimIndex[0], stimuli_means.columns.get_indexer(['means'])[0]]

responses

,seriousness_mean
0,1.42222
1,5.97778
2,6.35556
3,4.13333
4,1.57778
...,...
595,4.34
596,5.94
597,1.784
598,1.604


In [76]:
X_train, X_test, y_train, y_test = train_test_split(features, responses, test_size=0.2, random_state=0)

# Linear regression

In [82]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train) 
coeff_df = pd.DataFrame(regressor.coef_[0], features.columns, columns=['Coefficient'])  
print(coeff_df)

lm_score = regressor.score(X_train,y_train)
lm_score_test = regressor.score(X_test,y_test)

print(lm_score)
print(lm_score_test)

                                                Coefficient
actual_physical_injury                             0.259021
threat_of_death                                    1.726314
actual_death                                       2.046261
sexual_content                                     1.569297
physical_pain                                      0.789796
human_perpetrator                                  0.741647
close_int._perpetrator                            -0.447185
close_int._victim                                  0.622440
loss_of_possessions                                1.908679
threat_to_moral_identity                           1.562703
threat_to_social_status_or_other_identity          2.122851
threat_to_moral_worldview                          0.238857
threat_to_trust_in_interpersonal_relationships     1.356037
learned_about                                     -0.871191
witnessed                                          0.216620
child                                   

# Lasso

In [84]:
regressorLasso = LassoCV()  
regressorLasso.fit(X_train, y_train.values.ravel()) 
coeff_df_Lasso = pd.DataFrame(regressorLasso.coef_, features.columns, columns=['Coefficient'])  
print(coeff_df_Lasso)

# R^2
lasso_score = regressorLasso.score(X_train,y_train)
lasso_score_test = regressorLasso.score(X_test,y_test)

print(lasso_score)
print(lasso_score_test)

                                                Coefficient
actual_physical_injury                             0.247200
threat_of_death                                    1.690163
actual_death                                       1.962338
sexual_content                                     1.434211
physical_pain                                      0.790574
human_perpetrator                                  0.745027
close_int._perpetrator                            -0.200175
close_int._victim                                  0.526911
loss_of_possessions                                1.771161
threat_to_moral_identity                           1.409487
threat_to_social_status_or_other_identity          2.042184
threat_to_moral_worldview                          0.272291
threat_to_trust_in_interpersonal_relationships     1.168181
learned_about                                     -0.808822
witnessed                                          0.207092
child                                   

# Ridge regression

In [87]:
regressorRidge = RidgeCV()  
regressorRidge.fit(X_train, y_train.values.ravel()) 
coeff_df_Ridge = pd.DataFrame(regressorRidge.coef_, features.columns, columns=['Coefficient'])  
print(coeff_df_Ridge)

# R^2
ridge_score = regressorRidge.score(X_train,y_train)
ridge_score_test = regressorRidge.score(X_test,y_test)

print(ridge_score)
print(ridge_score_test)

                                                Coefficient
actual_physical_injury                             0.353642
threat_of_death                                    1.575953
actual_death                                       1.832563
sexual_content                                     1.335806
physical_pain                                      0.766715
human_perpetrator                                  0.795979
close_int._perpetrator                            -0.303627
close_int._victim                                  0.561341
loss_of_possessions                                1.645323
threat_to_moral_identity                           1.355663
threat_to_social_status_or_other_identity          1.890043
threat_to_moral_worldview                          0.291827
threat_to_trust_in_interpersonal_relationships     1.197921
learned_about                                     -0.808682
witnessed                                          0.246135
child                                   

# Load Binary Ratings Data

In [6]:
experiment1 = pd.read_csv(r"Experiment1_Data_Clean.csv")
experiment2 = pd.read_csv(r"Experiment2_Data_Clean.csv")
binData = pd.concat([experiment1,experiment2], axis=0)
binData.columns.values

array(['ResponseID', 'trial', 'StartDate', 'EndDate', 'DurationInSeconds',
       'RecordedDate', 'MentalDisorderTF', 'MentalDiorderList',
       'MentalDisorderOther', 'TraumaScreen', 'Gender', 'GenderOther',
       'Race', 'RaceOther', 'Ethnicity', 'Religiosity',
       'PoliticalOrientation', 'Age', 'toofast', 'condition',
       'attentionCheckFail', 'response', 'stimulus', 'norm_mean'],
      dtype=object)

# Define features for classification models

In [51]:
categoriesExtra = ['ResponseID', 'trial', 'DurationInSeconds','MentalDisorderTF','TraumaScreen',\
                  'Gender','Race','Ethnicity','Religiosity','PoliticalOrientation','Age',
                  'condition']

categories = ['actual_physical_injury', 'threat_of_death', 'actual_death', 'sexual_content',\
             'physical_pain','human_perpetrator','close_int._perpetrator',\
             'close_int._victim','loss_of_possessions','threat_to_moral_identity',
             'threat_to_social_status_or_other_identity','threat_to_moral_worldview',
             'threat_to_trust_in_interpersonal_relationships']

tempDat = pd.concat([ratings.loc[:,'events'], features], axis=1)
binData = binData.merge(tempDat, axis=1)
binData
# # Add categories columns to binData
# for category in categories:
#     binData[category] = None

# # Fill out those categories by matching to the even column
# for i in tqdm(range(binData.shape[0])):
#     event = binData.iloc[i, binData.columns.get_indexer(['stimulus'])[0]]
#     for i in range(ratings.shape[0]):
#         if ratings.loc[i,'events'] == event:
#             eventIndex = i
#     extraColumns = features.iloc[eventIndex, features.columns.get_indexer(categories)]
#     binData.iloc[i,features.columns.get_indexer(categories)] = extraColumns

,events,actual_physical_injury,threat_of_death,actual_death,sexual_content,physical_pain,human_perpetrator,close_int._perpetrator,close_int._victim,loss_of_possessions,threat_to_moral_identity,threat_to_social_status_or_other_identity,threat_to_moral_worldview,threat_to_trust_in_interpersonal_relationships,learned_about,witnessed,child
0,witnessed a leaf falling from a tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,lost a foot due to frostbite,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0
2,witnessed a stranger being raped,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,0
3,was tackled by a security officer,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,visited family members in another country,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,was disowned from a community,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0
596,was struck by lightning,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
597,moved to a different house in the same city,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
598,smoked a cigarette,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [40]:

for i in range(ratings.shape[0]):
    if ratings.loc[i,'events'] == "read a book in which an adult was injured in an explosion":
        eventIndex = i
extraColumns = features.iloc[eventIndex, features.columns.get_indexer(categories)]
extraColumns

actual_physical_injury                            1.0
threat_of_death                                   1.0
actual_death                                      0.0
sexual_content                                    0.0
physical_pain                                     1.0
human_perpetrator                                 0.0
close_int._perpetrator                            0.0
close_int._victim                                 0.0
loss_of_possessions                               0.0
threat_to_moral_identity                          0.0
threat_to_social_status_or_other_identity         0.0
threat_to_moral_worldview                         0.0
threat_to_trust_in_interpersonal_relationships    0.0
Name: 421, dtype: float64

In [49]:
features

,actual_physical_injury,threat_of_death,actual_death,sexual_content,physical_pain,human_perpetrator,close_int._perpetrator,close_int._victim,loss_of_possessions,threat_to_moral_identity,threat_to_social_status_or_other_identity,threat_to_moral_worldview,threat_to_trust_in_interpersonal_relationships,learned_about,witnessed,child
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0
2,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1,0
3,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0
596,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
